<a href="https://colab.research.google.com/github/sreent/machine-learning/blob/main/Decision%20Tree%20Classification/Decision%20Tree%20Classification%20Hands-On%20Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Tree Classification: Hands-On Lab

## Learning Objectives

By the end of this lab, you will be able to:

1. **Understand** how decision trees recursively partition feature space using greedy splits
2. **Implement** entropy and information gain calculations from scratch
3. **Build** a complete decision tree classifier using recursive tree construction
4. **Apply** decision trees to classification problems with both numerical and categorical features
5. **Evaluate** model performance using accuracy, confusion matrix, and cross-validation
6. **Tune** hyperparameters (max_depth, min_samples_split, min_samples_leaf) to control overfitting
7. **Visualize** decision boundaries and tree structure
8. **Interpret** feature importances based on information gain contributions

## Algorithm Overview

**Decision Trees** are supervised learning algorithms that learn a series of if/else questions to partition the feature space into regions, each associated with a class label.

### Key Concepts

**Entropy** measures the impurity or disorder in a dataset:

$$E = -\sum_{i=1}^{N} p_i \log_2(p_i)$$

Where:
- $p_i$ is the proportion of samples belonging to class $i$
- $N$ is the number of classes
- By convention, $0 \cdot \log_2(0) = 0$

**Information Gain** measures the reduction in entropy after a split:

$$IG = E_{parent} - \sum_{j} \frac{|S_j|}{|S|} E_j$$

Where:
- $E_{parent}$ is the entropy before the split
- $S_j$ is the subset of samples in child node $j$
- $E_j$ is the entropy of child node $j$

**Greedy Splitting**: At each node, the algorithm:
1. Considers all possible features and thresholds
2. Selects the split that maximizes information gain
3. Recursively builds subtrees until stopping criteria are met

---

### Why These Components?

**Why entropy?** Entropy quantifies uncertainty. A pure node (all samples from one class) has entropy 0, while a maximally mixed node has high entropy. By minimizing entropy, we create purer, more predictive leaf nodes.

**Why information gain?** It measures how much a split reduces uncertainty. Choosing splits with maximum information gain ensures we ask the most informative questions first, leading to shorter, more efficient trees.

**Why greedy splitting?** Finding the globally optimal tree is NP-hard. Greedy selection (locally optimal splits) is computationally tractable and works well in practice, though it may not find the globally optimal tree.

## When to Use Decision Trees

Decision trees are versatile classifiers with specific strengths and limitations. Understanding when to use them is crucial for effective model selection.

### ✅ Use Decision Trees When:

**1. Interpretability is Critical**
- Trees can be visualized and explained as a series of if/else rules
- Essential in healthcare, finance, and legal domains where decisions must be justified
- Example: "IF credit_score > 700 AND income > 50000 THEN approve_loan"

**2. Mixed Feature Types**
- Handles both numerical and categorical features natively
- No need for feature scaling or normalization
- No need for one-hot encoding (with proper implementation)

**3. Non-Linear Decision Boundaries**
- Captures complex, axis-aligned decision boundaries
- Automatically handles feature interactions through hierarchical splits
- Example: "IF age > 30 AND (income > 60000 OR education = 'PhD')"

**4. Feature Selection is Needed**
- Naturally performs feature selection via information gain
- Important features appear near the root; irrelevant features are ignored
- Feature importances are directly interpretable

**5. Quick Baseline Model**
- Fast to train and predict
- Requires minimal data preprocessing
- Good starting point before trying complex models

### ❌ Don't Use Decision Trees When:

**1. Smooth Decision Boundaries Required**
- Trees create axis-aligned, "staircase" boundaries
- Diagonal or curved boundaries require many splits
- **Better alternatives**: SVM with RBF kernel, Neural Networks, Logistic Regression

**2. High Variance is Unacceptable**
- Single trees are unstable—small data changes can produce very different trees
- **Better alternatives**: Random Forests, Gradient Boosting (ensembles of trees)

**3. Very High-Dimensional Sparse Data**
- Text data with thousands of features can lead to overfitting
- **Better alternatives**: Naive Bayes, Linear SVM, regularized models

**4. Extrapolation Beyond Training Data**
- Trees cannot extrapolate; predictions outside training range use nearest leaf
- **Better alternatives**: Linear models, Neural Networks

### Quick Comparison: Decision Trees vs Other Classifiers

| Criterion | Decision Tree | Logistic Regression | kNN | Naive Bayes | Random Forest |
|-----------|--------------|---------------------|-----|-------------|---------------|
| **Interpretability** | ✅ Excellent | ✅ Good | ❌ Poor | ✅ Good | ⚠️ Moderate |
| **Non-linear boundaries** | ✅ Good | ⚠️ Manual features | ✅ Excellent | ❌ Linear | ✅ Excellent |
| **Training speed** | ✅ Fast | ✅ Fast | ✅ Fast (lazy) | ✅ Very Fast | ⚠️ Moderate |
| **Prediction speed** | ✅ Very Fast | ✅ Very Fast | ❌ Slow | ✅ Very Fast | ⚠️ Moderate |
| **Handles missing data** | ✅ Yes | ❌ No | ❌ No | ⚠️ Partial | ✅ Yes |
| **Feature scaling needed** | ❌ No | ✅ Yes | ✅ Yes | ❌ No | ❌ No |
| **Stability** | ❌ Low | ✅ High | ⚠️ Moderate | ✅ High | ✅ High |
| **Overfitting risk** | ⚠️ High | ⚠️ Low | ⚠️ Moderate | ⚠️ Low | ⚠️ Low |

### Real-World Applications Where Decision Trees Excel:

1. **Medical Diagnosis**: Interpretable rules for disease detection
2. **Credit Risk Assessment**: Explainable loan approval decisions
3. **Customer Segmentation**: Clear rules for marketing targeting
4. **Fraud Detection**: Transparent reasoning for flagged transactions
5. **Manufacturing Quality Control**: Interpretable defect classification

## Pseudocode for Decision Tree Classifier

```
# Decision Tree Classifier — Greedy Recursive Splitting
# Inputs
# X, y        ← data (N×D features, N labels)
# max_depth   ← maximum tree depth
# min_leaf    ← minimum samples required in a leaf
# X_query     ← examples to predict

# ----- fit -----
Define BUILD(X, y, depth):
    IF all labels in y equal OR depth = max_depth OR |y| < 2·min_leaf THEN
        RETURN Leaf( class = majority(y) )
    
    N ← |y|
    base ← − Σ_k p_k log2 p_k          # parent entropy; p_k = freq(y=k)/N
    best_gain ← 0 ; best ← NONE
    
    FOR j = 1 TO D DO                   # iterate over features
        thresholds ← midpoints of sorted unique X[:, j]
        FOR each t IN thresholds DO
            L ← {i : X[i, j] ≤ t}       # left child indices
            R ← {i : X[i, j] > t}       # right child indices
            IF |L| < min_leaf OR |R| < min_leaf THEN CONTINUE
            H_L ← − Σ_k p_Lk log2 p_Lk  # left entropy
            H_R ← − Σ_k p_Rk log2 p_Rk  # right entropy
            gain ← base − (|L|/N)·H_L − (|R|/N)·H_R  # information gain
            IF gain > best_gain THEN 
                best_gain ← gain ; best ← (j, t, L, R)
    
    IF best = NONE THEN
        RETURN Leaf( class = majority(y) )
    
    (j*, t*, L, R) ← best
    left  ← BUILD(X[L, :], y[L], depth + 1)
    right ← BUILD(X[R, :], y[R], depth + 1)
    RETURN Node(feature=j*, threshold=t*, left=left, right=right)

tree ← BUILD(X, y, 0)

# ----- predict -----
ŷ ← list of length |X_query|
FOR i = 1 TO |X_query| DO
    node ← tree
    WHILE node is not Leaf DO
        IF X_query[i][node.feature] ≤ node.threshold THEN 
            node ← node.left
        ELSE 
            node ← node.right
    END WHILE
    ŷ[i] ← node.class
END FOR
RETURN ŷ
```

**Note:** The algorithm uses **greedy splitting** — at each node, it picks the single best split without looking ahead. This is computationally efficient but may not find the globally optimal tree.

## Hyperparameters

Decision trees have several key hyperparameters that control model complexity:

- **`max_depth`**: Maximum depth of the tree. Limits how many levels of splits are allowed.
  - Small values → simpler trees, may underfit
  - Large values → complex trees, may overfit

- **`min_samples_split`**: Minimum number of samples required to create a split (decision rule).
  - Large values → fewer splits, simpler trees
  - Small values → more splits, complex trees

- **`min_samples_leaf`**: Minimum number of samples required to be in a leaf node.
  - Large values → larger leaves, smoother decision boundaries
  - Small values → smaller leaves, more granular boundaries

**Trade-off**: These parameters control the **bias-variance trade-off**:
- **Underfitting** (high bias): Tree is too simple, misses patterns
- **Overfitting** (high variance): Tree memorizes noise, poor generalization

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier as SklearnDecisionTreeClassifier
from sklearn.tree import plot_tree
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## Exercise 1: Implement Entropy Calculation

Welcome to the hands-on implementation! We'll build the `MyDecisionTreeClassifier` class in **three independent exercises** to help you test and debug each component.

**In this exercise, you'll implement:**
- `_entropy()`: Calculate the entropy of a set of labels

**Entropy Formula:**
$$E = -\sum_{i=1}^{N} p_i \log_2(p_i)$$

**Why separate exercises?**
- Test each component immediately
- No cascading failures
- Build confidence step-by-step
- Easy to debug if something goes wrong

In [ ]:
class MyDecisionTreeClassifier(BaseEstimator, ClassifierMixin):
    """
    Custom Decision Tree Classifier using entropy and information gain.
    
    Parameters
    ----------
    max_depth : int, default=None
        Maximum depth of the tree. None means unlimited depth.
    min_samples_split : int, default=2
        Minimum number of samples required to split a node.
    min_samples_leaf : int, default=1
        Minimum number of samples required to be in a leaf node.
    random_state : int, default=None
        Random seed for reproducibility.
    """
    
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state
    
    def _entropy(self, y):
        """
        Calculate the entropy of a label array.
        
        Parameters
        ----------
        y : array-like
            Array of class labels (integers)
        
        Returns
        -------
        entropy : float
            Entropy value (0 = pure, higher = more impure)
        
        Formula
        -------
        E = -Σ p_i * log2(p_i) where p_i is the proportion of class i
        
        Notes
        -----
        - By convention, 0 * log2(0) = 0
        - Use np.bincount to count occurrences of each class
        """
        # TODO: Count occurrences of each class
        # Hint: Use np.bincount(y, minlength=self.n_classes_) if available,
        # or handle the general case
        if len(y) == 0:
            return 0.0
        
        # TODO: Calculate the proportion of each class
        # Hint: counts / total_samples
        counts = None
        
        # TODO: Calculate entropy using the formula: -Σ p_i * log2(p_i)
        # Hint: Only include classes with p > 0 to avoid log(0)
        # Use np.log2 for logarithm base 2
        entropy = None
        
        return entropy

In [ ]:
print("=" * 70)
print("EXERCISE 1 VERIFICATION: Testing Entropy Calculation")
print("=" * 70)

# Create a test instance
model_ex1 = MyDecisionTreeClassifier()
model_ex1.n_classes_ = 2  # Set for testing

print("\n1. Testing entropy on pure sets (should be 0):")
print("-" * 70)

# Pure set - all class 0
y_pure_0 = np.array([0, 0, 0, 0, 0])
entropy_pure_0 = model_ex1._entropy(y_pure_0)
print(f"All class 0: {y_pure_0} → Entropy = {entropy_pure_0:.4f}")
print(f"Expected: 0.0000")

# Pure set - all class 1
y_pure_1 = np.array([1, 1, 1, 1, 1])
entropy_pure_1 = model_ex1._entropy(y_pure_1)
print(f"All class 1: {y_pure_1} → Entropy = {entropy_pure_1:.4f}")
print(f"Expected: 0.0000")

print("\n2. Testing entropy on maximally mixed set (should be 1.0 for binary):")
print("-" * 70)

# Perfectly balanced binary
y_balanced = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
entropy_balanced = model_ex1._entropy(y_balanced)
print(f"Balanced (5 vs 5): {y_balanced} → Entropy = {entropy_balanced:.4f}")
print(f"Expected: 1.0000")

print("\n3. Testing entropy on imbalanced sets:")
print("-" * 70)

# 9 muffins, 1 cookie (from lecture slides)
y_low_entropy = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
entropy_low = model_ex1._entropy(y_low_entropy)
print(f"9 vs 1: {y_low_entropy} → Entropy = {entropy_low:.4f}")
print(f"Expected: ~0.4690 (from lecture: 0.47)")

# 6 blue, 5 orange (from lecture slides numerical example)
y_mixed = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
entropy_mixed = model_ex1._entropy(y_mixed)
print(f"6 vs 5: → Entropy = {entropy_mixed:.4f}")
print(f"Expected: ~0.9940 (from lecture: 0.994)")

print("\n4. Testing entropy on multiclass (3 classes):")
print("-" * 70)
model_ex1.n_classes_ = 3

# 3 muffins, 4 cookies, 2 cakes (from lecture example)
y_multiclass = np.array([0, 0, 0, 1, 1, 1, 1, 2, 2])
entropy_multi = model_ex1._entropy(y_multiclass)
print(f"3 vs 4 vs 2: → Entropy = {entropy_multi:.4f}")
print(f"Expected: ~1.5300 (from lecture: 1.52)")

print("\n" + "=" * 70)
print("If your outputs match the expected values, proceed to Exercise 2!")
print("=" * 70)

> **Question**: What does entropy measure in the context of decision trees?
>
> A) The total number of samples in a node, used to determine when to stop splitting the tree
>
> B) The impurity or disorder of class labels in a node, where zero means all samples belong to one class
>
> C) The depth of the current node from the root, controlling the maximum complexity of the tree
>
> D) The information gain achieved by splitting on a particular feature at the current node

<details>
<summary>Click to see answer</summary>

**Answer: B**

**Key Insight:** Entropy measures how "mixed" or "impure" the class labels are in a node. A pure node (entropy = 0) contains only one class, while a maximally impure node has equal proportions of all classes.

**Detailed Explanation:**

Entropy quantifies uncertainty:
- **E = 0**: All samples belong to one class (perfect purity, no uncertainty)
- **E = 1**: For binary classification, 50-50 split (maximum uncertainty)
- **E = log₂(N)**: Maximum entropy for N equally distributed classes

**Example with numbers:**
- 10 muffins, 0 cookies: E = -(10/10)log₂(10/10) = 0 (pure)
- 5 muffins, 5 cookies: E = -2×(5/10)log₂(5/10) = 1.0 (maximum impurity)
- 9 muffins, 1 cookie: E = -(9/10)log₂(9/10) - (1/10)log₂(1/10) ≈ 0.47 (low impurity)

**Why other answers are incorrect:**

- **A is FALSE**: Sample count is not entropy. The number of samples in a node affects the weighted average in information gain calculations, but entropy specifically measures class distribution impurity, not size.
- **C is FALSE**: Depth is a separate concept from entropy. Depth limits tree complexity through the `max_depth` hyperparameter, while entropy measures label purity at a specific node regardless of its depth.
- **D is FALSE**: This describes information gain, not entropy. Information gain is computed as the difference between parent entropy and the weighted sum of child entropies. Entropy is a prerequisite for calculating information gain.

</details>

## Exercise 2: Implement Information Gain Calculation

Excellent! You now have a working entropy function. Let's implement **information gain** — the criterion for selecting the best split.

**What you'll implement:**
- `_information_gain()`: Calculate the reduction in entropy from a split

**Information Gain Formula:**
$$IG = E_{parent} - \frac{|L|}{N} E_L - \frac{|R|}{N} E_R$$

**Why this matters:**
Information gain tells us how much a split reduces uncertainty. We always choose the split with the highest information gain.

In [ ]:
# Add information gain method to MyDecisionTreeClassifier
def _information_gain(self, y_parent, y_left, y_right):
    """
    Calculate information gain from a split.
    
    Parameters
    ----------
    y_parent : array-like
        Labels before the split
    y_left : array-like
        Labels in the left child after split
    y_right : array-like
        Labels in the right child after split
    
    Returns
    -------
    gain : float
        Information gain (higher = better split)
    
    Formula
    -------
    IG = E_parent - (|L|/N * E_L + |R|/N * E_R)
    """
    n_parent = len(y_parent)
    n_left = len(y_left)
    n_right = len(y_right)
    
    if n_left == 0 or n_right == 0:
        return 0.0
    
    # TODO: Calculate parent entropy using your _entropy method
    e_parent = None
    
    # TODO: Calculate left and right child entropies
    e_left = None
    e_right = None
    
    # TODO: Calculate weighted average of child entropies
    # weighted_child_entropy = (n_left/n_parent) * e_left + (n_right/n_parent) * e_right
    weighted_child_entropy = None
    
    # TODO: Calculate information gain = parent entropy - weighted child entropy
    gain = None
    
    return gain

# Add the method to the class
MyDecisionTreeClassifier._information_gain = _information_gain

> **Question**: Why do we use weighted average of child entropies in information gain?
>
> A) To give equal importance to both child nodes regardless of their size in the calculation
>
> B) To penalize splits that create very small child nodes which might contain noise
>
> C) To account for the proportion of samples going to each child, since larger subsets have more influence
>
> D) To ensure the information gain is always positive and bounded between zero and one

<details>
<summary>Click to see answer</summary>

**Answer: C**

**Key Insight:** Weighted averaging accounts for how many samples flow into each child node. A split that creates one very pure but tiny child and one large impure child isn't necessarily good—we care about the overall reduction in uncertainty across all samples.

**Detailed Explanation:**

The weighted formula is:
$$IG = E_{parent} - \frac{|L|}{N} E_L - \frac{|R|}{N} E_R$$

**Example:**
- Parent: 100 samples, E = 1.0
- Split A: Left (99 samples, E=0.8), Right (1 sample, E=0)
  - Weighted: (99/100)×0.8 + (1/100)×0 = 0.792
  - IG = 1.0 - 0.792 = 0.208
- Split B: Left (50 samples, E=0.5), Right (50 samples, E=0.5)
  - Weighted: (50/100)×0.5 + (50/100)×0.5 = 0.5
  - IG = 1.0 - 0.5 = 0.5

Split B is better because it reduces uncertainty across more samples, even though Split A created a "perfect" (but tiny) right child.

**Why other answers are incorrect:**

- **A is FALSE**: Equal weighting would overvalue tiny pure nodes. A split creating 1 pure sample vs 99 mixed samples would appear artificially good.
- **B is FALSE**: While weighted averaging does de-emphasize tiny nodes, this is a consequence of proper probability weighting, not an explicit penalty. The formula doesn't directly penalize small nodes.
- **D is FALSE**: Information gain is guaranteed to be non-negative (you can't increase entropy by splitting), but it's not bounded by 1. For multiclass problems, the maximum possible gain equals the parent entropy, which can exceed 1.

</details>

In [ ]:
print("=" * 70)
print("EXERCISE 2 VERIFICATION: Testing Information Gain Calculation")
print("=" * 70)

# Create test instance
model_ex2 = MyDecisionTreeClassifier()
model_ex2.n_classes_ = 2

print("\n1. Testing information gain (from lecture slides - Latte/Mocha example):")
print("-" * 70)

# Parent: 12 muffins, 5 cookies, 2 cakes → 19 total
# After split on Drink:
#   Latte: 9 muffins, 1 cookie → 10 samples (low entropy)
#   Mocha: 3 muffins, 4 cookies, 2 cakes → 9 samples (high entropy)

# Simplified binary case from lecture: E_parent = 0.994 (6 blue, 5 orange)
y_parent = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1])  # 6 vs 5

# Split at x1 = 1.0: Left gets 5 blue, 1 orange; Right gets 1 blue, 4 orange
y_left = np.array([0, 0, 0, 0, 0, 1])   # 5 vs 1 → E ≈ 0.65
y_right = np.array([0, 1, 1, 1, 1])      # 1 vs 4 → E ≈ 0.72

ig = model_ex2._information_gain(y_parent, y_left, y_right)
print(f"Parent: 6 blue, 5 orange (E ≈ 0.994)")
print(f"Left: 5 blue, 1 orange | Right: 1 blue, 4 orange")
print(f"Information Gain = {ig:.4f}")
print(f"Expected: ~0.31")

print("\n2. Testing perfect split (IG should equal parent entropy):")
print("-" * 70)

# Perfect split separates classes completely
y_parent_perfect = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
y_left_perfect = np.array([0, 0, 0, 0, 0])   # All class 0
y_right_perfect = np.array([1, 1, 1, 1, 1])  # All class 1

ig_perfect = model_ex2._information_gain(y_parent_perfect, y_left_perfect, y_right_perfect)
print(f"Perfect split: All 0s left, All 1s right")
print(f"Information Gain = {ig_perfect:.4f}")
print(f"Expected: 1.0000 (equals parent entropy)")

print("\n3. Testing no improvement split (IG should be 0):")
print("-" * 70)

# Split that doesn't improve purity
y_parent_same = np.array([0, 0, 1, 1, 0, 0, 1, 1])
y_left_same = np.array([0, 0, 1, 1])   # 50-50
y_right_same = np.array([0, 0, 1, 1])  # 50-50

ig_same = model_ex2._information_gain(y_parent_same, y_left_same, y_right_same)
print(f"No improvement: Both children have same distribution as parent")
print(f"Information Gain = {ig_same:.4f}")
print(f"Expected: 0.0000")

print("\n4. Testing lecture example (Seating split):")
print("-" * 70)

# From slide 14: Seating split on café dataset
# Parent: 10 muffins, 9 cookies, 6 cakes = 25 samples, E = 1.554
model_ex2.n_classes_ = 3
y_cafe_parent = np.array([0]*10 + [1]*9 + [2]*6)  # muffin=0, cookie=1, cake=2

# Indoor: 10 muffins, 0 cookies, 6 cakes = 16 samples
y_indoor = np.array([0]*10 + [2]*6)
# Outdoor: 0 muffins, 9 cookies, 0 cakes = 9 samples (pure!)
y_outdoor = np.array([1]*9)

ig_seating = model_ex2._information_gain(y_cafe_parent, y_indoor, y_outdoor)
print(f"Seating split: Indoor (10 muffin, 6 cake) | Outdoor (9 cookie)")
print(f"Information Gain = {ig_seating:.4f}")
print(f"Expected: ~0.943 (from lecture slide 14)")

print("\n" + "=" * 70)
print("If your outputs match the expected values, proceed to Exercise 3!")
print("=" * 70)

## Exercise 3: Implement Full Decision Tree Classifier

Excellent! You now have working components:
- ✅ Entropy calculation
- ✅ Information gain calculation

Now let's put it all together and implement the **complete decision tree classifier**!

**What you'll implement:**
- `_find_best_split()`: Find the best feature and threshold to split on
- `_build_tree()`: Recursively build the tree
- `fit()`: Train the model
- `predict()` and `predict_proba()`: Make predictions

**The tree building process:**
1. Check stopping conditions (pure node, max depth, min samples)
2. Find the best split (feature + threshold with highest information gain)
3. Split the data and recursively build left and right subtrees
4. Return a leaf node if no good split is found

In [ ]:
# Complete the MyDecisionTreeClassifier class
def _find_best_split(self, X, y):
    """
    Find the best feature and threshold to split on.
    
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        Feature matrix
    y : array-like, shape (n_samples,)
        Labels
    
    Returns
    -------
    best_split : dict or None
        Dictionary with 'feature', 'threshold', 'left_mask', 'right_mask', 'gain'
        Returns None if no valid split is found
    """
    n_samples, n_features = X.shape
    best_gain = 0.0
    best_split = None
    
    # TODO: Iterate over all features
    for feature_idx in range(n_features):
        # Get unique values and compute midpoint thresholds
        feature_values = X[:, feature_idx]
        unique_values = np.unique(feature_values)
        
        if len(unique_values) < 2:
            continue
        
        # TODO: Calculate midpoint thresholds between consecutive unique values
        # Hint: thresholds = (unique_values[:-1] + unique_values[1:]) / 2
        thresholds = None
        
        # TODO: Try each threshold
        for threshold in thresholds:
            # TODO: Create masks for left (<=) and right (>) children
            left_mask = None
            right_mask = None
            
            # Check minimum samples constraint
            n_left = np.sum(left_mask)
            n_right = np.sum(right_mask)
            
            if n_left < self.min_samples_leaf or n_right < self.min_samples_leaf:
                continue
            
            # TODO: Calculate information gain for this split
            y_left = y[left_mask]
            y_right = y[right_mask]
            gain = None  # Use self._information_gain(y, y_left, y_right)
            
            # TODO: Update best split if this is better
            if gain > best_gain:
                best_gain = gain
                best_split = {
                    'feature': feature_idx,
                    'threshold': threshold,
                    'left_mask': left_mask,
                    'right_mask': right_mask,
                    'gain': gain
                }
    
    return best_split

def _build_tree(self, X, y, depth=0):
    """
    Recursively build the decision tree.
    
    Parameters
    ----------
    X : array-like
        Feature matrix for current node
    y : array-like
        Labels for current node
    depth : int
        Current depth in the tree
    
    Returns
    -------
    node : dict
        Either a leaf node {'leaf': True, 'proba': [...], 'class': c}
        or an internal node {'leaf': False, 'feature': f, 'threshold': t, 'left': ..., 'right': ...}
    """
    n_samples = len(y)
    
    # Calculate class probabilities for this node
    counts = np.bincount(y, minlength=self.n_classes_)
    proba = counts / counts.sum()
    majority_class = np.argmax(counts)
    
    # TODO: Check stopping conditions
    # 1. All samples belong to the same class (pure node)
    # 2. Reached maximum depth (if max_depth is set)
    # 3. Not enough samples to split (< min_samples_split)
    
    # Condition 1: Pure node
    if len(np.unique(y)) == 1:
        return {'leaf': True, 'proba': proba, 'class': majority_class}
    
    # Condition 2: Max depth reached
    if self.max_depth is not None and depth >= self.max_depth:
        return {'leaf': True, 'proba': proba, 'class': majority_class}
    
    # Condition 3: Not enough samples
    if n_samples < self.min_samples_split:
        return {'leaf': True, 'proba': proba, 'class': majority_class}
    
    # TODO: Find the best split
    best_split = self._find_best_split(X, y)
    
    # No valid split found
    if best_split is None:
        return {'leaf': True, 'proba': proba, 'class': majority_class}
    
    # TODO: Recursively build left and right subtrees
    left_tree = self._build_tree(
        X[best_split['left_mask']], 
        y[best_split['left_mask']], 
        depth + 1
    )
    right_tree = self._build_tree(
        X[best_split['right_mask']], 
        y[best_split['right_mask']], 
        depth + 1
    )
    
    return {
        'leaf': False,
        'feature': best_split['feature'],
        'threshold': best_split['threshold'],
        'left': left_tree,
        'right': right_tree,
        'gain': best_split['gain']
    }

def fit(self, X, y):
    """
    Build the decision tree from training data.
    
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        Training features
    y : array-like, shape (n_samples,)
        Training labels
    
    Returns
    -------
    self : object
        Returns self for method chaining
    """
    X = np.asarray(X, dtype=float)
    y = np.asarray(y)
    
    # Store class information
    self.classes_, y_encoded = np.unique(y, return_inverse=True)
    self.n_classes_ = len(self.classes_)
    self.n_features_ = X.shape[1]
    
    # Build the tree
    self.tree_ = self._build_tree(X, y_encoded, depth=0)
    
    return self

def _predict_single(self, x, node):
    """
    Predict class probabilities for a single sample by traversing the tree.
    """
    # TODO: Traverse tree until reaching a leaf
    while not node['leaf']:
        if x[node['feature']] <= node['threshold']:
            node = node['left']
        else:
            node = node['right']
    
    return node['proba']

def predict_proba(self, X):
    """
    Predict class probabilities.
    
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        Samples to predict
    
    Returns
    -------
    proba : array, shape (n_samples, n_classes)
        Class probabilities for each sample
    """
    X = np.asarray(X, dtype=float)
    probas = np.array([self._predict_single(x, self.tree_) for x in X])
    return probas

def predict(self, X):
    """
    Predict class labels.
    
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        Samples to predict
    
    Returns
    -------
    y_pred : array, shape (n_samples,)
        Predicted class labels
    """
    proba = self.predict_proba(X)
    return self.classes_[np.argmax(proba, axis=1)]

# Add all methods to the class
MyDecisionTreeClassifier._find_best_split = _find_best_split
MyDecisionTreeClassifier._build_tree = _build_tree
MyDecisionTreeClassifier.fit = fit
MyDecisionTreeClassifier._predict_single = _predict_single
MyDecisionTreeClassifier.predict_proba = predict_proba
MyDecisionTreeClassifier.predict = predict

> **Question**: What is the greedy approach in decision tree learning?
>
> A) The algorithm considers all possible tree structures and selects the globally optimal one
>
> B) At each node, it selects the locally best split without considering future consequences
>
> C) The algorithm prunes the tree after building it to remove unnecessary branches
>
> D) It randomly selects features and thresholds to create diverse decision boundaries

<details>
<summary>Click to see answer</summary>

**Answer: B**

**Key Insight:** Greedy algorithms make the locally optimal choice at each step. In decision trees, this means selecting the split with the highest information gain at the current node, without considering how this choice affects future splits.

**Detailed Explanation:**

At each node, the algorithm:
1. Evaluates all possible (feature, threshold) combinations
2. Selects the one with maximum information gain
3. Never backtracks or reconsiders previous decisions

**Trade-off:**
- **Advantage**: Computationally efficient (polynomial time vs NP-hard for global optimization)
- **Disadvantage**: May not find the globally optimal tree

**Example:**
A greedy split on feature A might look best now, but splitting on feature B first could lead to a better overall tree. Greedy algorithms can't "see" this.

**Why other answers are incorrect:**

- **A is FALSE**: Finding the globally optimal decision tree is NP-hard. Greedy algorithms don't consider all possible trees—they build incrementally, making local decisions.
- **C is FALSE**: This describes post-pruning, a technique to prevent overfitting. Pruning is separate from the greedy splitting process and typically uses validation data.
- **D is FALSE**: This describes Random Forests, not standard decision trees. Random Forests intentionally introduce randomness to create diverse trees for ensemble learning.

</details>

In [ ]:
print("=" * 70)
print("EXERCISE 3 VERIFICATION: Testing Complete Decision Tree")
print("=" * 70)

# Test data from lecture slides (numerical features example)
X_test = np.array([
    [-0.5, -4.0], [-1.5, -2.5], [0.0, 0.0], [-1.0, 0.5], [0.5, 1.5], [2.5, 1.0],
    [3.5, -3.5], [2.0, -3.0], [3.0, -2.0], [1.5, -1.5], [4.0, -1.0]
])
y_test = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

print("\n1. Training on lecture example data:")
print("-" * 70)

model_ex3 = MyDecisionTreeClassifier(max_depth=3, min_samples_leaf=1)
model_ex3.fit(X_test, y_test)

print(f"✓ Training completed")
print(f"✓ Number of classes: {model_ex3.n_classes_}")
print(f"✓ Classes: {model_ex3.classes_}")

print("\n2. Testing predictions:")
print("-" * 70)

y_pred = model_ex3.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"True labels:      {y_test}")
print(f"Predicted labels: {y_pred}")
print(f"Training Accuracy: {accuracy:.2%}")

print("\n3. Testing single prediction (from lecture - point (2, -2)):")
print("-" * 70)

X_new = np.array([[2.0, -2.0]])
proba = model_ex3.predict_proba(X_new)[0]
pred = model_ex3.predict(X_new)[0]

print(f"New Point: {X_new[0].tolist()}")
print(f"Probabilities [class 0, class 1]: {proba.tolist()}")
print(f"Predicted Class: {pred}")
print(f"Expected Class: 1 (from lecture slide 41)")

print("\n4. Tree structure check:")
print("-" * 70)

def print_tree(node, indent=0):
    prefix = "  " * indent
    if node['leaf']:
        print(f"{prefix}Leaf: class={node['class']}, proba={node['proba']}")
    else:
        print(f"{prefix}Node: X[{node['feature']}] <= {node['threshold']:.3f} (gain={node['gain']:.3f})")
        print(f"{prefix}  Left:")
        print_tree(node['left'], indent + 2)
        print(f"{prefix}  Right:")
        print_tree(node['right'], indent + 2)

print_tree(model_ex3.tree_)

print("\n" + "=" * 70)
print("SUCCESS! Your complete implementation is working!")
print("=" * 70)
print("\nYou can now proceed to apply your model to larger datasets!")

## Load Classification Dataset

We'll use the **mixture dataset from the lecture slides (slides 43-52)**, which is also used in the Regularisation and Logistic Regression labs. This dataset contains two classes with non-linear separation, making it perfect for demonstrating:
1. How decision trees partition feature space with axis-aligned splits
2. The bias-variance trade-off with tree depth
3. How hyperparameters control model complexity

In [ ]:
# Download the mixture dataset from Google Drive
# File ID: 1Ls7f9OWKgeWswFR4EZ5eeoohfY9PACRT
# Direct download URL
url = 'https://drive.google.com/uc?id=1Ls7f9OWKgeWswFR4EZ5eeoohfY9PACRT'

# Load data
df = pd.read_csv(url)
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nColumn names: {df.columns.tolist()}")

# Extract features and labels (last column is the label)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values.astype(int)

print(f"\nFeatures shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Class distribution: {np.bincount(y)}")

## Visualize the Data

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(X[y == 0, 0], X[y == 0, 1], c='orange', label='Class 0',
            edgecolors='k', s=50, alpha=0.7)
plt.scatter(X[y == 1, 0], X[y == 1, 1], c='skyblue', label='Class 1',
            edgecolors='k', s=50, alpha=0.7)
plt.xlabel('$x_1$', fontsize=14)
plt.ylabel('$x_2$', fontsize=14)
plt.title('Mixture Dataset (Non-Linear Boundary)', fontsize=16)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

## Split into Train and Test Sets

In [ ]:
# Split data (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Train class distribution: {np.bincount(y_train)}")
print(f"Test class distribution: {np.bincount(y_test)}")

## Train the Model

In [ ]:
# Create and train model
model = MyDecisionTreeClassifier(max_depth=3, min_samples_split=2, min_samples_leaf=1)
model.fit(X_train, y_train)

print("Training completed!")
print(f"Number of classes: {model.n_classes_}")

## Make Predictions and Evaluate

In [ ]:
# Predict on test set
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)

# Evaluate
train_accuracy = accuracy_score(y_train, model.predict(X_train))
test_accuracy = accuracy_score(y_test, y_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Show some predictions
print("\nSample predictions:")
for i in range(min(5, len(y_test))):
    print(f"True: {y_test[i]}, Predicted: {y_pred[i]}, P(class=1): {y_proba[i, 1]:.3f}")

## Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
print("\n[TN  FP]")
print("[FN  TP]")

# Visualize
plt.figure(figsize=(6, 5))
plt.imshow(cm, cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
for i in range(2):
    for j in range(2):
        plt.text(j, i, cm[i, j], ha='center', va='center', fontsize=20)
plt.xticks([0, 1], ['Class 0', 'Class 1'])
plt.yticks([0, 1], ['Class 0', 'Class 1'])
plt.show()

## Classification Report

In [ ]:
print(classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1']))

## Visualize Decision Boundary

Decision trees create axis-aligned ("staircase") decision boundaries. Let's visualize how our tree partitions the feature space.

In [ ]:
def plot_decision_boundary(model, X, y, title="Decision Boundary"):
    """Plot decision boundary for a 2D classification problem."""
    # Create mesh
    x1_min, x1_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    x2_min, x2_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 200),
                           np.linspace(x2_min, x2_max, 200))
    
    # Predict on mesh
    X_mesh = np.c_[xx1.ravel(), xx2.ravel()]
    if hasattr(model, 'predict_proba'):
        Z = model.predict_proba(X_mesh)[:, 1].reshape(xx1.shape)
    else:
        Z = model.predict(X_mesh).reshape(xx1.shape)
    
    # Plot
    plt.figure(figsize=(10, 8))
    plt.contourf(xx1, xx2, Z, levels=20, cmap='RdBu_r', alpha=0.6)
    plt.colorbar(label='P(Class 1)')
    plt.contour(xx1, xx2, Z, levels=[0.5], colors='black', linewidths=2)
    
    # Plot data points
    plt.scatter(X[y == 0, 0], X[y == 0, 1], c='orange', label='Class 0', 
                edgecolors='k', s=50, alpha=0.7)
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c='skyblue', label='Class 1', 
                edgecolors='k', s=50, alpha=0.7)
    
    plt.xlabel('$x_1$', fontsize=14)
    plt.ylabel('$x_2$', fontsize=14)
    plt.title(title, fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.show()

plot_decision_boundary(model, X, y, "Decision Tree Decision Boundary (max_depth=3)")

## K-Fold Cross-Validation

So far we've evaluated our model using a single train/test split. Let's use K-fold cross-validation for a more robust performance estimate.

In [ ]:
# Use training data for cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(
    MyDecisionTreeClassifier(max_depth=3, min_samples_split=2, min_samples_leaf=1),
    X_train, y_train, cv=kf, scoring='accuracy'
)

print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV Accuracy: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")

## Hyperparameter Tuning: Effect of max_depth

Let's explore how different values of `max_depth` affect model performance and the bias-variance trade-off.

In [ ]:
depths = [1, 2, 3, 4, 5, None]  # None means unlimited
train_scores = []
val_scores = []

for depth in depths:
    model_depth = MyDecisionTreeClassifier(max_depth=depth, min_samples_split=2, min_samples_leaf=1)
    
    # Training score
    model_depth.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, model_depth.predict(X_train))
    train_scores.append(train_acc)
    
    # Cross-validation score
    cv_scores = cross_val_score(model_depth, X_train, y_train, cv=5, scoring='accuracy')
    val_scores.append(np.mean(cv_scores))
    
    depth_str = str(depth) if depth is not None else "None"
    print(f"max_depth={depth_str:>4}: Train={train_acc:.3f}, CV={np.mean(cv_scores):.3f}")

# Plot
plt.figure(figsize=(10, 6))
depth_labels = [str(d) if d is not None else 'None' for d in depths]
x_pos = range(len(depths))
plt.plot(x_pos, train_scores, 'o-', label='Train Accuracy', linewidth=2, markersize=8)
plt.plot(x_pos, val_scores, 's-', label='CV Accuracy', linewidth=2, markersize=8)
plt.xticks(x_pos, depth_labels)
plt.xlabel('max_depth', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Effect of max_depth on Model Performance', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

## Visualize Effect of Hyperparameters on Decision Boundary

Let's see how different `min_samples_split` values affect the decision boundary.

In [ ]:
min_samples_values = [2, 4, 8, 12]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

for idx, min_samples in enumerate(min_samples_values):
    ax = axes[idx]
    
    # Train model
    model_ms = MyDecisionTreeClassifier(max_depth=None, min_samples_split=min_samples, min_samples_leaf=1)
    model_ms.fit(X_train, y_train)
    
    # Cross-validation score
    cv_scores = cross_val_score(model_ms, X_train, y_train, cv=5, scoring='accuracy')
    
    # Create mesh
    x1_min, x1_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    x2_min, x2_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 150),
                           np.linspace(x2_min, x2_max, 150))
    
    X_mesh = np.c_[xx1.ravel(), xx2.ravel()]
    Z = model_ms.predict_proba(X_mesh)[:, 1].reshape(xx1.shape)
    
    # Plot
    ax.contourf(xx1, xx2, Z, levels=20, cmap='RdBu_r', alpha=0.6)
    ax.contour(xx1, xx2, Z, levels=[0.5], colors='black', linewidths=2)
    ax.scatter(X_train[y_train == 0, 0], X_train[y_train == 0, 1], 
               c='orange', edgecolors='k', s=40, alpha=0.7)
    ax.scatter(X_train[y_train == 1, 0], X_train[y_train == 1, 1], 
               c='skyblue', edgecolors='k', s=40, alpha=0.7)
    ax.set_title(f'min_samples_split = {min_samples}, CV Acc = {np.mean(cv_scores):.2f}', fontsize=12)
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Understanding the Bias-Variance Trade-off

From the visualizations above:

- **Small min_samples_split (e.g., 2)**: Complex boundaries, may overfit to noise
  - High variance, low bias
  - High train accuracy, potentially lower validation accuracy

- **Large min_samples_split (e.g., 12)**: Simple boundaries, may underfit
  - Low variance, high bias  
  - Lower train and validation accuracy

- **Optimal**: Balance between complexity and generalization

## Feature Importances

Decision trees provide natural feature importances based on how much each feature contributes to reducing entropy (information gain) across all splits.

**How it works:**
1. At each split, calculate: `weighted_gain = (n_samples_at_node / n_total) × information_gain`
2. Sum these contributions for each feature
3. Normalize so all importances sum to 1

In [ ]:
def compute_feature_importances(model):
    """
    Compute feature importances based on weighted information gain.
    """
    importances = np.zeros(model.n_features_)
    
    def traverse(node, n_samples):
        if node['leaf']:
            return
        
        # Weight the gain by the proportion of samples at this node
        weight = n_samples / len(y_train)
        importances[node['feature']] += weight * node['gain']
        
        # Estimate samples in children (rough approximation)
        n_left = n_samples // 2
        n_right = n_samples - n_left
        
        traverse(node['left'], n_left)
        traverse(node['right'], n_right)
    
    traverse(model.tree_, len(y_train))
    
    # Normalize
    if importances.sum() > 0:
        importances = importances / importances.sum()
    
    return importances

# Train a fresh model
model_fi = MyDecisionTreeClassifier(max_depth=3, min_samples_split=2, min_samples_leaf=1)
model_fi.fit(X_train, y_train)

# Compute importances
importances = compute_feature_importances(model_fi)

print("Feature Importances:")
for i, imp in enumerate(importances):
    print(f"  Feature X[{i}]: {imp:.4f} ({imp*100:.1f}%)")

# Visualize
plt.figure(figsize=(8, 5))
plt.bar(['$X_1$', '$X_2$'], importances, color=['steelblue', 'coral'], edgecolor='black')
plt.ylabel('Importance', fontsize=12)
plt.title('Feature Importances', fontsize=14)
plt.ylim(0, 1)
for i, imp in enumerate(importances):
    plt.text(i, imp + 0.02, f'{imp:.2%}', ha='center', fontsize=12)
plt.show()

## Comparison with scikit-learn

Let's validate our implementation by comparing it with sklearn's `DecisionTreeClassifier`.

In [ ]:
# Train sklearn model
sklearn_model = SklearnDecisionTreeClassifier(
    criterion='entropy',
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)
sklearn_model.fit(X_train, y_train)

# Compare accuracies
our_train_acc = accuracy_score(y_train, model_fi.predict(X_train))
our_test_acc = accuracy_score(y_test, model_fi.predict(X_test))
sklearn_train_acc = sklearn_model.score(X_train, y_train)
sklearn_test_acc = sklearn_model.score(X_test, y_test)

print("Performance Comparison:")
print("-" * 50)
print(f"{'Model':<20} {'Train Acc':<15} {'Test Acc':<15}")
print("-" * 50)
print(f"{'Our Implementation':<20} {our_train_acc:<15.4f} {our_test_acc:<15.4f}")
print(f"{'sklearn':<20} {sklearn_train_acc:<15.4f} {sklearn_test_acc:<15.4f}")

print("\nFeature Importances Comparison:")
print("-" * 50)
print(f"{'Feature':<15} {'Our Model':<15} {'sklearn':<15}")
print("-" * 50)
for i in range(model_fi.n_features_):
    print(f"{'X[' + str(i) + ']':<15} {importances[i]:<15.4f} {sklearn_model.feature_importances_[i]:<15.4f}")

## Visualize sklearn Decision Tree

In [ ]:
plt.figure(figsize=(16, 10))
plot_tree(sklearn_model, 
          feature_names=['$X_1$', '$X_2$'], 
          class_names=['Class 0', 'Class 1'],
          filled=True, 
          rounded=True,
          fontsize=10)
plt.title('sklearn Decision Tree Structure', fontsize=14)
plt.tight_layout()
plt.show()

## Best Practices and Tips

### 1. Controlling Overfitting
- **max_depth**: Limit tree depth to prevent overly complex trees
- **min_samples_split**: Require minimum samples to create a split
- **min_samples_leaf**: Require minimum samples in each leaf
- **Use cross-validation** to select optimal hyperparameters

### 2. Feature Handling
- **No scaling required**: Trees are invariant to monotonic transformations
- **Categorical features**: Use label encoding (as shown in lecture slides)
- **Missing values**: Some implementations handle them; consider imputation for custom code

### 3. Interpretability
- **Visualize the tree**: Use `sklearn.tree.plot_tree` or export to graphviz
- **Extract rules**: Convert tree to if-else statements
- **Feature importances**: Identify most predictive features

### 4. When Trees Struggle
- **Linear boundaries**: Logistic regression may be simpler and better
- **High variance**: Consider Random Forests (ensemble of trees)
- **Extrapolation**: Trees cannot extrapolate beyond training data range

### 5. Evaluation
- **Use stratified splits** for imbalanced classes
- **Report multiple metrics**: Accuracy, precision, recall, F1-score
- **Visualize decision boundaries** to understand model behavior

## Summary

In this lab, you:

1. ✅ **Implemented** entropy calculation to measure node impurity
2. ✅ **Implemented** information gain to evaluate split quality
3. ✅ **Built** a complete decision tree classifier from scratch using recursive partitioning
4. ✅ **Trained** the model on synthetic classification data
5. ✅ **Evaluated** performance using accuracy, confusion matrix, and cross-validation
6. ✅ **Visualized** decision boundaries showing axis-aligned splits
7. ✅ **Tuned** hyperparameters (max_depth, min_samples_split) and observed bias-variance trade-off
8. ✅ **Computed** feature importances based on information gain contributions
9. ✅ **Compared** your implementation with scikit-learn

### Key Takeaways

- **Entropy** measures impurity: $E = -\sum p_i \log_2(p_i)$
- **Information Gain** is the reduction in entropy: $IG = E_{parent} - \text{weighted avg}(E_{children})$
- **Greedy splitting** selects the locally best split at each node
- **Hyperparameters** control the bias-variance trade-off:
  - Small values → complex trees, potential overfitting
  - Large values → simple trees, potential underfitting
- **Feature importances** show which features contribute most to predictions
- **No feature scaling** is required for decision trees
- **Decision boundaries** are axis-aligned (rectangular regions)

### Next Steps

- Apply decision trees to real-world datasets (e.g., Iris, Wine, Breast Cancer)
- Implement Gini impurity as an alternative to entropy
- Learn about Random Forests (ensembles of trees) to reduce variance
- Explore Gradient Boosted Trees (XGBoost, LightGBM) for state-of-the-art performance
- Implement pruning techniques to prevent overfitting